# Order By

Example: Which day of the week has the most fatal motor accidents?
Let's use the US Traffic Fatality Records database, which contains information on traffic accidents in the US where at least one person died.

We'll investigate the accident_2015 table. Here is a view of the first few rows. (We have hidden the corresponding code. To take a peek, click on the "Code" button below.)

In [1]:
from google.cloud import bigquery
client = bigquery.Client.from_service_account_json("./My Project 61217-9183df10355d.json")
dataset_ref = client.dataset('nhtsa_traffic_fatalities', project = 'bigquery-public-data')
# API request -fetch dataset
dataset = client.get_dataset(dataset_ref)

In [2]:
# list all the tables in the hacker news dataset
tables = list(client.list_tables(dataset))
for table in tables:
    print(table.table_id)

accident_2015
accident_2016
cevent_2015
cevent_2016
damage_2015
damage_2016
distract_2015
distract_2016
drimpair_2015
drimpair_2016
factor_2015
factor_2016
maneuver_2015
maneuver_2016
nmcrash_2015
nmcrash_2016
nmimpair_2015
nmimpair_2016
nmprior_2015
nmprior_2016
parkwork_2015
parkwork_2016
pbtype_2015
pbtype_2016
person_2015
person_2016
safetyeq_2015
safetyeq_2016
vehicle_2015
vehicle_2016
vevent_2015
vevent_2016
vindecode_2015
vindecode_2016
violatn_2015
violatn_2016
vision_2015
vision_2016
vsoe_2015
vsoe_2016


In [3]:
table_ref = dataset_ref.table('accident_2015')
table = client.get_table(table_ref)

In [4]:
client.list_rows(table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00


Let's use the table to determine how the number of accidents varies with the day of the week. Since:

the consecutive_number column contains a unique ID for each accident, and
the timestamp_of_crash column contains the date of the accident in DATETIME format,
we can:

EXTRACT the day of the week (as day_of_week in the query below) from the timestamp_of_crash column, and
GROUP BY the day of the week, before we COUNT the consecutive_number column to determine the number of accidents for each day of the week.
Then we sort the table with an ORDER BY clause, so the days with the most accidents are returned first.

In [5]:
query = """
        SELECT Count(consecutive_number) AS num_accidents
                EXTRACT (DAYOFWEEK FROM timestamp_of_crash column) AS day_of_week
        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
        GROUP BY day_of_week
        ORDER BY num_accidents DESC
        """

In [7]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day

BadRequest: 400 Syntax error: Unexpected keyword EXTRACT at [3:17]

(job ID: a74fa250-22da-4a65-bc53-883f49fec06c)

                           -----Query Job SQL Follows-----                            

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:
   2:        SELECT Count(consecutive_number) AS num_accidents
   3:                EXTRACT (DAYOFWEEK FROM timestamp_of_crash column) AS day_of_week
   4:        FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
   5:        GROUP BY day_of_week
   6:        ORDER BY num_accidents DESC
   7:        
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |